In [1]:
!pip install pyspark

In [46]:
#import necessary packages
from pyspark.sql import SparkSession

#create spark context
sqlCtx = SparkSession.builder.getOrCreate()

from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType

Load the data from the csv files into DataFrames.

In [17]:
crashes = sqlCtx.read.option('header','true').csv('data/Traffic_Crashes_-_Crashes.csv')
vehicles = sqlCtx.read.option('header','true').csv('data/Traffic_Crashes_-_Vehicles.csv')
peoples = sqlCtx.read.option('header','true').csv('data/Traffic_Crashes_-_People.csv')
print(type(crashes))
peoples.dtypes


<class 'pyspark.sql.dataframe.DataFrame'>


[('PERSON_ID', 'string'),
 ('PERSON_TYPE', 'string'),
 ('CRASH_RECORD_ID', 'string'),
 ('RD_NO', 'string'),
 ('VEHICLE_ID', 'string'),
 ('CRASH_DATE', 'string'),
 ('SEAT_NO', 'string'),
 ('CITY', 'string'),
 ('STATE', 'string'),
 ('ZIPCODE', 'string'),
 ('SEX', 'string'),
 ('AGE', 'string'),
 ('DRIVERS_LICENSE_STATE', 'string'),
 ('DRIVERS_LICENSE_CLASS', 'string'),
 ('SAFETY_EQUIPMENT', 'string'),
 ('AIRBAG_DEPLOYED', 'string'),
 ('EJECTION', 'string'),
 ('INJURY_CLASSIFICATION', 'string'),
 ('HOSPITAL', 'string'),
 ('EMS_AGENCY', 'string'),
 ('EMS_RUN_NO', 'string'),
 ('DRIVER_ACTION', 'string'),
 ('DRIVER_VISION', 'string'),
 ('PHYSICAL_CONDITION', 'string'),
 ('PEDPEDAL_ACTION', 'string'),
 ('PEDPEDAL_VISIBILITY', 'string'),
 ('PEDPEDAL_LOCATION', 'string'),
 ('BAC_RESULT', 'string'),
 ('BAC_RESULT VALUE', 'string'),
 ('CELL_PHONE_USE', 'string')]

Find the ratio of number of crashes where the person involved was using cell phone to that where the person was not using the cell phone.

In [28]:
peoples.groupby(peoples.DRIVER_ACTION).count().orderBy("count").show(truncate=False)

+---------------------------------+-----+
|DRIVER_ACTION                    |count|
+---------------------------------+-----+
|EMERGENCY VEHICLE ON CALL        |3    |
|TEXTING                          |3    |
|CELL PHONE USE OTHER THAN TEXTING|5    |
|EVADING POLICE VEHICLE           |5    |
|OVERCORRECTED                    |7    |
|IMPROPER PARKING                 |16   |
|WRONG WAY/SIDE                   |21   |
|TOO FAST FOR CONDITIONS          |49   |
|IMPROPER LANE CHANGE             |69   |
|DISREGARDED CONTROL DEVICES      |73   |
|IMPROPER TURN                    |92   |
|IMPROPER PASSING                 |99   |
|IMPROPER BACKING                 |125  |
|null                             |176  |
|FOLLOWED TOO CLOSELY             |184  |
|OTHER                            |241  |
|FAILED TO YIELD                  |291  |
|NONE                             |1073 |
|UNKNOWN                          |1507 |
+---------------------------------+-----+



In [37]:
phone = peoples.groupby(peoples.DRIVER_ACTION).count().filter((peoples.DRIVER_ACTION == 'CELL PHONE USE OTHER THAN TEXTING') | (peoples.DRIVER_ACTION == 'TEXTING'))
phone.show(truncate=False)
phone_crashes = phone.groupBy().sum('count').collect()[0][0]
print("crashes that occurs because of phone : ", phone_crashes)

+---------------------------------+-----+
|DRIVER_ACTION                    |count|
+---------------------------------+-----+
|CELL PHONE USE OTHER THAN TEXTING|5    |
|TEXTING                          |3    |
+---------------------------------+-----+

crashes that occurs because of phone :  8


In [38]:
no_phone = peoples.where((peoples.DRIVER_ACTION != "UNKNOWN") & (peoples.DRIVER_ACTION != "NONE") & (peoples.DRIVER_ACTION != "OTHER"))\
.groupby("DRIVER_ACTION").count().filter((peoples.DRIVER_ACTION != 'CELL PHONE USE OTHER THAN TEXTING') & (peoples.DRIVER_ACTION != 'TEXTING'))

no_phone.show(truncate=False)
no_phone_crashes = no_phone.groupBy().sum('count').collect()[0][0]
print("crashes that occurs because of other : ", no_phone_crashes)

+---------------------------+-----+
|DRIVER_ACTION              |count|
+---------------------------+-----+
|EVADING POLICE VEHICLE     |5    |
|FOLLOWED TOO CLOSELY       |184  |
|IMPROPER LANE CHANGE       |69   |
|IMPROPER PARKING           |16   |
|TOO FAST FOR CONDITIONS    |49   |
|DISREGARDED CONTROL DEVICES|73   |
|IMPROPER TURN              |92   |
|IMPROPER BACKING           |125  |
|OVERCORRECTED              |7    |
|WRONG WAY/SIDE             |21   |
|IMPROPER PASSING           |99   |
|FAILED TO YIELD            |291  |
|EMERGENCY VEHICLE ON CALL  |3    |
+---------------------------+-----+

crashes that occurs because of other :  1034


In [39]:
print("ratio: ", f"{phone_crashes}/{no_phone_crashes} =", 100 * phone_crashes/no_phone_crashes)

ratio:  8/1034 = 0.7736943907156673


*bring in package to convert age code to int for sorting

In [47]:
age = peoples.where((peoples.AGE > 0)).groupby(peoples.AGE.cast(IntegerType())).count().orderBy("count", ascending=False).limit(3)
age.show(120, truncate=False)

+----------------+-----+
|CAST(AGE AS INT)|count|
+----------------+-----+
|30              |78   |
|25              |77   |
|28              |76   |
+----------------+-----+



In [61]:
month = crashes.select((crashes.CRASH_MONTH).cast(IntegerType()))\
                    .groupby("CRASH_MONTH")\
                    .count()\
                    .orderBy("count", ascending=False)
month.show(truncate=False)


+-----------+-----+
|CRASH_MONTH|count|
+-----------+-----+
|5          |1195 |
|6          |600  |
|4          |493  |
|12         |8    |
|9          |8    |
|10         |7    |
|8          |6    |
|1          |6    |
|3          |6    |
|2          |6    |
|11         |5    |
|7          |3    |
|null       |1    |
+-----------+-----+



In [64]:
df = crashes.select((crashes.CRASH_DAY_OF_WEEK).cast(IntegerType()))\
                    .groupby("CRASH_DAY_OF_WEEK")\
                    .count()\
                    .orderBy("count", ascending=True)
df.show(truncate=False)

+-----------------+-----+
|CRASH_DAY_OF_WEEK|count|
+-----------------+-----+
|null             |1    |
|6                |148  |
|5                |229  |
|2                |236  |
|1                |274  |
|7                |285  |
|3                |577  |
|4                |594  |
+-----------------+-----+

